In [1]:
import os
import glob
import numpy as np

def get_all_alpha_ry(label_root):
    labels = glob.glob(os.path.join(label_root, '*.txt'), recursive=True)
    Alphas = list()
    Rys = list()
    for i in range(len(labels)):
        with open(labels[i]) as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                elements = line[:-1].split()
                if elements[0] == 'DontCare':
                    continue
                alpha = float(elements[3])
                ry = float(elements[14])
                Alphas.append(alpha)
                Rys.append(ry)
    Alphas = np.array(Alphas)
    Rys = np.array(Rys)
    return Alphas, Rys 

## GT part, run once and load npy in future

In [2]:
os.makedirs('orient_delta', exist_ok=True)
gt_alphas, gt_rys = get_all_alpha_ry(label_root='./Kitti/training/label_2') # len 40570
np.save('orient_delta/gt_alphas.npy', gt_alphas)
np.save('orient_delta/gt_rys.npy', gt_rys)
#gt_alpha = np.load('orient_delta/gt_alphas.npy')
#gt_ry = np.load('orient_delta/gt_rys.npy')

## Eval part, for generated pred labels

In [3]:
# best sum() 40570, mean() 1
def angle_criterion(PRED, GT):
    cos_delta = np.cos(PRED - GT)
    return cos_delta
    #return (1+cos_delta)/2

In [7]:
eval_folder = 'multi_bin_yolo_kitti20'
os.makedirs(f'orient_delta/{eval_folder}', exist_ok=True)
eval_alphas, eval_rys = get_all_alpha_ry(label_root=eval_folder) # len 40570
np.save(f'orient_delta/{eval_folder}/eval_alphas.npy', eval_alphas)
np.save(f'orient_delta/{eval_folder}/eval_rys.npy', eval_rys)
#eval_alphas = np.load(f'orient_delta/{eval_folder}/eval_alphas.npy')
#eval_rys = np.load(f'orient_delta/{eval_folder}/eval_rys.npy')
delta_alpha = angle_criterion(eval_alphas, gt_alphas)
delta_ry = angle_criterion(eval_rys, gt_rys)
print(f'Alpha sum:{delta_alpha.sum()}, mean:{delta_alpha.mean()}')
print(f'Ry sum:{delta_ry.sum()}, mean:{delta_ry.mean()}')

ValueError: operands could not be broadcast together with shapes (41758,) (40570,) 